<a href="https://colab.research.google.com/github/bbberylll/ESAA_OB/blob/main/09_05_%EC%84%B8%EC%85%98_%EB%B6%84%EB%A5%98_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **분류 연습 문제**
___
출처 : 핸즈온 머신러닝 Ch03 분류 연습문제 1, 2번

In [1]:
# import data
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1, as_frame = False)

In [2]:
X, y = mnist["data"], mnist["target"]

In [3]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### **1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.**
___

1. `KNeghtborsClassifier`를 사용하는 것을 추천합니다.
2. `weights`와 `n_neighbors` 하이퍼 파라미터로 그리드 탐색을 시도하여, 좋은 하이퍼 파라미터 값을 찾아보세요.

In [4]:
# import package
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
# Try GridSearch to optimize hyperparameter

kk = KNeighborsClassifier()

parameters = {'weights':['distance', 'uniform'], 'n_neighbors':[3,4,5]}

grid = GridSearchCV(kk, param_grid=parameters, cv = 3, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)

In [ ]:
# best hyperparameter
grid.best_params_

In [ ]:
# best score
grid.best_score_

In [ ]:
# model test
y_pred = grid.predict(X_test)
print("정확도 : ", accuracy_score(y_test, y_pred))

### **2. 다음 단계를 따라 인위적으로 훈련 세트를 늘리는 데이터 증식 또는 훈련 세트 확장 기법을 연습해봅시다.**
___

#### **STEP 1. MNIST 이미지를 (왼, 오른, 위, 아래) 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어 보세요.**

## Shift 함수

데이터 프레임의 행을 위, 아래로 옮기고 싶을 때 사용하면 유용함
근데 이건 시계열에서 쓰이는거 같은디...?  흠



In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
from scipy.ndimage import shift
## 이미지 증강에 쓰이는 함수 -- 다차원 배열을 주어진 양만큼 공간적으로 이동시킴

def shift_image(image, dx, dy):
    image=image.reshape((28, 28))
    shifted_image=shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])


####  **STEP 2. 앞에서 만든 함수를 이용하여, 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본(방향마다 한 개씩)을 만들어 훈련 세트에 추가하세요**

In [ ]:
image=X_train[1000]
shifted_image_down=shift_image(image, 0, 5)
shifted_image_up=shift_image(image, 0, -1)
shifted_image_left=shift_image(image, -5, 0)
shifted_image_right=shift_image(image, 1, 0)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,3))

plt.subplot(1,5,1)
plt.title("Original", fontsize=14)
plt.imshow(image.reshape(28, 28), interpolation="nearest", cmap="Greys")

plt.subplot(1,5,2)
plt.title("Shifted down", fontsize=14)
plt.imshow(shifted_image_down.reshape(28, 28), interpolation="nearest", cmap="Greys")

plt.subplot(1,5,3)
plt.title("Shifted up", fontsize=14)
plt.imshow(shifted_image_up.reshape(28, 28), interpolation="nearest", cmap="Greys")

plt.subplot(1,5,4)
plt.title("Shifted left", fontsize=14)
plt.imshow(shifted_image_left.reshape(28, 28), interpolation="nearest", cmap="Greys")

plt.subplot(1,5,5)
plt.title("Shifted right", fontsize=14)
plt.imshow(shifted_image_right.reshape(28, 28), interpolation="nearest", cmap="Greys")

plt.show()

In [ ]:

X_train_augmented=[image for image in X_train]
y_train_augmented=[label for label in y_train]

for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train_augmented=np.array(X_train_augmented)
y_train_augmented=np.array(y_train_augmented)


shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

## 개선 지점

1. 라벨 증식은 y 복사로 진행해야 한다고 함.
x에서 적용한 이동 함수를 적용하는 것이 아닌 이미지만 이동하고 라벨은 그대로 복제해야 한다고 한다.

2. shift는 3차 보간이라 숫자 이미지가 블러될 수 있다고 한다. 정확한 의미는 추가적인 공부를 통해 확인해야 할 듯.
따라서 order=0 + mode='constant', cva=l=0 으로 지정하면 픽셀이 무너지지 않는다고 한다

3. 배치 이동은 패딩 + 슬라이싱으로 빠르게 진행할 수 있다고 하낟.
shift는 루프를 호출할 경우 느려질 수 있다고 함

4. KNN은 거리 기반이기 때문에 스케일의 영향이 크다고 한다. 따라서 정규화를 권장함

5. 증식 후 셔플하는 것은 필수라고 한다.

####  **STEP 3. 위에서 확장한 데이터셋을 이용하여, 1번 문제에서 찾은 최적 모델을 훈련시키고, 테스트 세트에서 정확도를 측정해보세요**

In [ ]:
knn_clf=KNeighborsClassifier(**grid.best_params_)

In [ ]:
knn_clf.fit(X_train_augmented, y_train_augmented)

In [ ]:
y_pred=knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)